# Amor Reduction

## How to start:

Before starting you must:
- Have conda installed
- conda env create -f ess-notebooks-latest.yml python=3.7 . The yaml environment file is part of this repository.

## What will this notebook show?

The notebook will show how to use the `ess.amor.AmorData`, `ess.amor.AmorReference` and `ess.amor.Normalisation` classes for the reduction of data collected at the Amor instrument at PSI. 
To achieve this, we will reduce the following data files, [sample.nxs](./sample.nxs) and [reference.nxs](./references.nxs), and then normalise the sample data (with respect to the reference data). 

## Reduction

Before, we begin, we should import then necessary modules. 

In [1]:
import numpy as np
import scipp as sc
import scippneutron as scn
from ess.amor import AmorData, AmorReference, Normalisation

<IPython.core.display.Javascript object>

We can then define some information about the reduction, for inclusion in the final `.ort` file.

In [2]:
name = 'Andrew McCluskey'
affiliation = 'European Spallation Source'
data_owner = 'Jochen Stahn, PSI'
experiment_id = 'test_0001'
experiment_date = '2020-06-21'
sample_description = 'Ni-Ti Multilayer'
notebook_file = 'amor_reduction.ipynb'

data_file = 'sample.nxs'
reference_file = 'reference.nxs'

The `AmorData` class will take the loaded NeXus file or the NeXus file itself and perform the reduction steps to obtain the reflected intensity as a function of $q_z$, including accounting for aspects such as gravity. 
The `sample_angle_offset` allows the angular offset of the sample with respect to the horizon to be accounted for.

In [4]:
sample = AmorData(data_file, 
                  reduction_creator=name, 
                  data_owner=data_owner, 
                  experiment_id=experiment_id, 
                  experiment_date=experiment_date, 
                  sample_description=sample_description, 
                  reduction_file=notebook_file, 
                  reduction_creator_affiliation=affiliation,
                  sample_angle_offset=0.04 * sc.units.deg, 
                  sample_size=0.1*sc.units.m)

Some detector and wavelength masking can then be performed.

In [5]:
sample.detector_masking(y_min=0 * sc.units.m, y_max=100e-3 * sc.units.m)
sample.wavelength_masking()

The `AmorReference` class reads the reference supermirror measurement, and will perform the necessary corrections.
For this measurement, no angular offset is required. 
Again, detector and wavelength masking is performed.

In [6]:
reference = AmorReference(reference_file)
reference.detector_masking(y_min=0 * sc.units.m, y_max=100e-3 * sc.units.m)
reference.wavelength_masking()

For the normalisation of the sample, we use the `Normalisation` class.

In [7]:
norm = Normalisation(sample, reference)

With this object, there is the choice to bin in the $\lambda$/$\theta$-space or the $q_z$-space. 

In [8]:
bins = (np.linspace(2.5, 15, 50), np.linspace(0.6, 1.25, 50))
lambda_theta = norm.wavelength_theta_bin(bins)

The data binned in $\lambda$/$\theta$-space can be investigated and plotted.

In [9]:
lambda_theta

<scipp.DataArray>
Dimensions: {{wavelength, 49}, {theta, 49}}
Coordinates:
  theta                     float64    [deg]            (theta [bin-edge])  [0.600000, 0.613265, ..., 1.236735, 1.250000]
  wavelength                float64    [Å]             (wavelength [bin-edge])  [2.500000, 2.755102, ..., 14.744898, 15.000000]
Data:
                            float32    [dimensionless]  (wavelength, theta)  [0.029258, 0.039410, ..., 0.682027, 0.887515]  [0.000172, 0.000156, ..., 0.124597, 0.122891]
Attributes:
  experiment_title          string     [dimensionless]  ()  ["commissioning"]
  instrument_name           string     [dimensionless]  ()  ["AMOR"]
  sample_position           vector_3_float64  [m]              ()  [(0.000000, 0.000000, 0.000000)]
  source_position           vector_3_float64  [m]              ()  [(0.000000, 0.000000, -15.000000)]

In [10]:
sc.plot(lambda_theta, norm='log')

In [11]:
q_bins = np.linspace(0.007, 0.09, 200)
q = norm.q_bin(q_bins)

As can be done for the $q_z$-binned data.

In [12]:
fig = sc.plot(q, norm='log')
fig.ax.set_ylim((1e-3, 10))
fig

/Users/andrewmccluskey/anaconda3/envs/scipp_with_mantid/lib/python3.7/scipp/plotting/figure1d.py:353: UserWarning: Attempted to set non-positive bottom ylim on a log-scaled axis.
Invalid limit will be ignored.
  self.ax.set_ylim(vmin, vmax)


This data can be written to an [ORSO](https://reflectometry.org)-compatible data file. 

In [13]:
norm.write_reflectometry('output.ort', bin_kwargs={'bins': q_bins})

In [14]:
!head output.ort

# # ORSO reflectivity data file | 0.1 standard | YAML encoding | https://reflectometry.org
# creator:
#   name: Andrew McCluskey
#   time: '2021-04-21T13:39:01'
#   system: CI0021603
#   affiliation: European Spallation Source
# data_source:
#   experiment_date: '2020-06-21'
#   experiment:
#     instrument: AMOR
